# Voila를 이용한 프로젝트 시각화

In [9]:
import os
import glob
import numpy as np
import math
import time
import random
from PIL import Image
import cv2
import io
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2

import ipywidgets as widgets
from IPython.display import display

In [12]:
## model 불러오기
saved_path = './Model Part/data/saved/small_efficientnet_b7/epoch_20.pt'

## train 한대로 리스트를 정의해주세요.
celeb_list = [
    '공유', '권나라', '문채원', '박하선', '배두나', '신민아', '신성록', '안재현', '여진구', '유아인',
    '이다희', '이병헌', '이선균', '이시영', '전미도', '정은지', '조정석', '차승원', '한선화', '현빈',
]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## train 한대로 모델을 정의해주세요
model = torchvision.models.efficientnet_b7(pretrained = False)
model.classifier[1] = nn.Linear(in_features=2560, out_features=len(celeb_list), bias=True)

model.load_state_dict(torch.load(saved_path))
model.to(device)
model.eval()
print('')

In [13]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

test_transform = A.Compose([
                            A.Resize(256,256),
                            ToTensorV2()
                            ])

def get_prediction(model, content):
    global img
    # img = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
    img_ = np.frombuffer(content, dtype=np.uint8)
    img = cv2.imdecode(img_, cv2.IMREAD_COLOR)
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img /= 255.0
    
    transformed = test_transform(image=img)
    img = transformed['image']
    img = img.to(device)
    img = img.unsqueeze(0)
    
    pred = model(img)
    pred = pred.to('cpu').argmax()
    
    return pred

In [14]:
uploader = widgets.FileUpload(
    accept = '.jpg, .png, .jpeg',
    multiple=False,
)

---
### 원하는 배우 이미지를 업로드 해주세요.

In [15]:
display(uploader)

FileUpload(value={}, accept='.jpg, .png, .jpeg', description='Upload')

In [16]:
def on_display_click_callback(clicked_button: widgets.Button) -> None:
    global content
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    display_image_space.value = content

---
## 이미지를 확인해주세요.
### 확인해야 결과를 얻을 수 있습니다.

In [17]:
display_button = widgets.Button(description="Display Image")
display_image_space = widgets.Image()

display_button.on_click(on_display_click_callback)
display(display_button, display_image_space)

Button(description='Display Image', style=ButtonStyle())

Image(value=b'')

In [18]:
inference_button = widgets.Button(description="Inference!")
inference_output = widgets.Output(layout={'border': "1px solid black"})

In [19]:
def on_inference_click_callback(clicked_button: widgets.Button) -> None:
    with inference_output:
        inference_output.clear_output()

        start = time.time() # 시간 측정 시작
        output = get_prediction(model, content)
        result = celeb_list[output]
        print(result)
        print(f'Inference Time : {time.time() - start:.5f} second')

In [20]:
inference_button.on_click(on_inference_click_callback)

---
## 결과 확인

In [21]:
display(inference_button, inference_output)

Button(description='Inference!', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))